# Cleaning a PostgreSQL Database
![Clean PostgreSQL Database](Project_Image.jpeg)

In this project, you will work with data from a hypothetical Super Store to challenge and enhance your SQL skills in data cleaning. This project will engage you in identifying top categories based on the highest profit margins and detecting missing values, utilizing your comprehensive knowledge of SQL concepts.

## Data Dictionary:

### `orders`:
| Column | Definition | Data type | Comments |
|--------|------------|-----------|----------|
| `row_id`| Unique Record ID | `INTEGER` |
| `order_id` | Identifier for each order in table | `TEXT` | Connects to `order_id` in `returned_orders` table |
| `order_date` | Date when order was placed | `TEXT` |
| `market` | Market order_id belongs to | `TEXT` |
| `region` | Region Customer belongs to | `TEXT` | Connects to `region` in `people` table |
| `product_id` | Identifier of Product bought | `TEXT` | Connects to `product_id` in `products` table |
| `sales` | Total Sales Amount for the Line Item | `DOUBLE PRECISION` |
| `quantity` | Total Quantity for the Line Item | `DOUBLE PRECISION` |
| `discount` | Discount applied for the Line Item | `DOUBLE PRECISION` |
| `profit` | Total Profit earned on the Line Item | `DOUBLE PRECISION` |

### `returned_orders`:
| Column | Definition | Data type |
|--------|------------|-----------|
| `returned`| Yes values for Order / Line Item Returned | `TEXT` |
| `order_id` | Identifier for each order in table | `TEXT` |
| `market` | Market order_id belongs to | `TEXT` |

### `people`:
| Column | Definition | Data type |
|--------|------------|-----------|
| `person`| Name of Salesperson credited with Order | `TEXT` |
| `region` | Region Salesperson in operating in | `TEXT` |

### `products`:
| Column | Definition | Data type |
|--------|------------|-----------|
| `product_id`| Unique Identifier for the Product | `TEXT` |
| `category` | Category Product belongs to | `TEXT` |
| `sub_category` | Sub Category Product belongs to | `TEXT` |
| `product_name` | Detailed Name of the Product | `TEXT` |

### As you can see in the Data Dictionary above, date fields have been written to the `orders` table as `TEXT` and numeric fields like sales, profit, etc. have been written to the `orders` table as `Double Precision`. You will need to take care of these types in some of the queries. This project is an excellent opportunity to apply your SQL skills in a practical setting and gain valuable experience in data cleaning and analysis. Good luck, and happy querying!

In [14]:
-- top_five_products_each_category
WITH cte AS (
SELECT category, product_name, ROUND(SUM(sales::NUMERIC), 2) AS product_total_sales,
ROUND(SUM(profit::NUMERIC), 2) AS product_total_profit, RANK() OVER(PARTITION BY category ORDER BY SUM(sales) DESC) AS product_rank
FROM orders
INNER JOIN products USING(product_id)
GROUP BY category, product_name
ORDER BY category, product_total_sales DESC)

SELECT *
FROM cte 
WHERE product_rank < 6;

-- top_five_products_each_category
/*
SELECT *
FROM (
	SELECT category, product_name, ROUND(SUM(sales::NUMERIC), 2) AS product_total_sales,
	ROUND(SUM(profit::NUMERIC), 2) AS product_total_profit, RANK() OVER(PARTITION BY category ORDER BY SUM(sales) 		DESC) AS product_rank
	FROM orders
	INNER JOIN products USING(product_id)
	GROUP BY category, product_name
	ORDER BY category, product_total_sales DESC) AS sub
WHERE sub.product_rank < 6;
*/


,category,product_name,product_total_sales,product_total_profit,product_rank
0,Furniture,"Hon Executive Leather Armchair, Adjustable",58193.48,5997.25,1
1,Furniture,"Office Star Executive Leather Armchair, Adjust...",51449.80,4925.80,2
2,Furniture,"Harbour Creations Executive Leather Armchair, ...",50121.52,10427.33,3
3,Furniture,"SAFCO Executive Leather Armchair, Black",41923.53,7154.28,4
4,Furniture,"Novimex Executive Leather Armchair, Adjustable",40585.13,5562.35,5
5,Office Supplies,"Eldon File Cart, Single Width",39873.23,5571.26,1
6,Office Supplies,"Hoover Stove, White",32842.60,-2180.63,2
7,Office Supplies,"Hoover Stove, Red",32644.13,11651.68,3
8,Office Supplies,"Rogers File Cart, Single Width",29558.82,2368.82,4
9,Office Supplies,"Smead Lockers, Industrial",28991.66,3630.44,5


In [15]:
-- salesperson_market_sales_details

WITH cte_2 AS (
SELECT person, o.market, 
       (CASE WHEN o.sales >= 0 AND o.sales < 100	THEN '0-100'
		WHEN o.sales >= 100 AND o.sales < 500 THEN '100-500'
		WHEN o.sales >= 500 THEN '500+'
		END) AS sales_bin,
	   COUNT(order_id) AS order_counts, 
	   SUM(CASE WHEN returned = 'Yes' THEN 1 ELSE 0 END) AS orders_returned,
	   ROUND(SUM(sales::NUMERIC), 2) AS total_sales, 
	   ROUND(SUM(CASE WHEN returned = 'Yes' THEN sales::NUMERIC ELSE 0 END), 2) AS returned_sales
	   FROM orders AS o
	   JOIN people USING(region)
	   LEFT JOIN returned_orders USING(order_id)
GROUP BY 1, 2, 3
ORDER BY 1, 2, 3)

SELECT *
FROM cte_2;
/*

-- salesperson_market_sales_details
SELECT ppl.person,
	ord.market,
	CASE WHEN ord.sales >= 0 AND ord.sales < 100	THEN '0-100'
		WHEN ord.sales >= 100 AND ord.sales < 500 THEN '100-500'
		WHEN ord.sales >= 500 THEN '500+'
		END AS sales_bin,
		COUNT(DISTINCT ord.order_id) AS order_counts,
		SUM(CASE WHEN ret.returned IS NULL THEN 0 ELSE 1 END) AS orders_returned,
		SUM(ord.sales) AS total_sales,
		SUM(CASE WHEN ret.returned IS NULL THEN 0 ELSE ord.sales END) AS returned_sales
FROM orders as ord
INNER JOIN people AS ppl
	ON ord.region = ppl.region
LEFT JOIN returned_orders AS ret
	ON ord.order_id = ret.order_id AND ord.market = ret.market
GROUP BY sales_bin, ord.market, ppl.person
ORDER BY ppl.person, ord.market, sales_bin;
*/

,person,market,sales_bin,order_counts,orders_returned,total_sales,returned_sales
0,Alejandro Ballentine,APAC,0-100,1530,69,66548.57,3440.57
1,Alejandro Ballentine,APAC,100-500,1102,59,263646.16,15190.24
2,Alejandro Ballentine,APAC,500+,497,25,554228.44,25047.59
3,Anna Andreadi,EU,0-100,2553,159,123939.80,7951.06
4,Anna Andreadi,EU,100-500,2291,162,529997.12,37894.53
5,Anna Andreadi,EU,500+,978,61,1066615.70,61685.21
6,Anna Andreadi,LATAM,0-100,1712,80,70412.08,3059.10
7,Anna Andreadi,LATAM,100-500,944,45,217929.35,10767.64
8,Anna Andreadi,LATAM,500+,316,9,312168.58,9244.37
9,Anna Andreadi,US,0-100,1491,64,42592.42,1796.01


In [16]:
-- impute_missing_values
/* 
WITH missing AS (
    SELECT product_id, discount, market, region, sales, quantity,
           NULL::NUMERIC AS calculated_quantity
    FROM orders
    WHERE quantity IS NULL
),
unit_prices AS (
    SELECT product_id, discount,
           (sales / NULLIF(quantity, 0))::NUMERIC AS unit_price
    FROM orders
    WHERE quantity IS NOT NULL
)
SELECT m.product_id, m.discount, m.market, m.region, m.sales, m.quantity,
       ROUND((m.sales::NUMERIC / u.unit_price), 0) AS calculated_quantity
FROM missing m
INNER JOIN unit_prices u
ON m.product_id = u.product_id AND m.discount = u.discount;
*/

WITH missing AS (
	SELECT product_id,
		discount, 
		market,
		region,
		sales,
		quantity
	FROM orders 
	WHERE quantity IS NULL
), 

unit_prices AS (SELECT o.product_id,
	CAST(o.sales / o.quantity AS NUMERIC) AS unit_price
FROM orders o
RIGHT JOIN missing AS m 
	ON o.product_id = m.product_id
	AND o.discount = m.discount
WHERE o.quantity IS NOT NULL
)

SELECT DISTINCT m.*,
	ROUND(CAST(m.sales AS NUMERIC) / up.unit_price,0) AS calculated_quantity
FROM missing AS m
INNER JOIN unit_prices AS up
	ON m.product_id = up.product_id;

,product_id,discount,market,region,sales,quantity,calculated_quantity
0,FUR-ADV-10000571,0.00,EMEA,EMEA,438.960,NaN,4
1,FUR-ADV-10004395,0.00,EMEA,EMEA,84.120,NaN,2
2,FUR-BO-10001337,0.15,US,West,308.499,NaN,3
3,TEC-STA-10003330,0.00,Africa,Africa,506.640,NaN,2
4,TEC-STA-10004542,0.00,Africa,Africa,160.320,NaN,4
